In [13]:
# Stating all the packages here
from __future__  import print_function, division
import matplotlib.pylab as pl
import pandas as pd
import numpy as np
import os

%pylab inline

if os.getenv ('PUIDATA') is None:
    print ("Must set env variable PUIDATA")


import os
import json


Populating the interactive namespace from numpy and matplotlib


In [14]:
import pandas as pd
import numpy as py
import os
import sys
import zipfile

In [15]:
if os.getenv("PUIDATA") is None:
    os.environ["PUIDATA"] = '%s/Puidata'%os.getenv("HOME")

In [16]:
os.getenv('PUIDATA')

'/nfshome/kc116/PUIdata'

In [17]:
!curl https://s3.amazonaws.com/tripdata/201701-citibike-tripdata.csv.zip > Jan2017citibike.csv.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 23.1M  100 23.1M    0     0  40.3M      0 --:--:-- --:--:-- --:--:-- 40.3M


In [18]:
!mv Jan2017citibike.csv.zip $PUIDATA

mv: 'Jan2017citibike.csv.zip' and '/nfshome/kc116/PUIdata/Jan2017citibike.csv.zip' are the same file


In [20]:
df = pd.read_csv('Jan2017citibike.csv.zip')

In [21]:
df.head()

,Trip Duration,Start Time,Stop Time,Start Station ID,Start Station Name,Start Station Latitude,Start Station Longitude,End Station ID,End Station Name,End Station Latitude,End Station Longitude,Bike ID,User Type,Birth Year,Gender
0,680,2017-01-01 00:00:21,2017-01-01 00:11:41,3226,W 82 St & Central Park West,40.782750,-73.971370,3165,Central Park West & W 72 St,40.775794,-73.976206,25542,Subscriber,1965.0,2
1,1282,2017-01-01 00:00:45,2017-01-01 00:22:08,3263,Cooper Square & E 7 St,40.729236,-73.990868,498,Broadway & W 32 St,40.748549,-73.988084,21136,Subscriber,1987.0,2
2,648,2017-01-01 00:00:57,2017-01-01 00:11:46,3143,5 Ave & E 78 St,40.776829,-73.963888,3152,3 Ave & E 71 St,40.768737,-73.961199,18147,Customer,NaN,0
3,631,2017-01-01 00:01:10,2017-01-01 00:11:42,3143,5 Ave & E 78 St,40.776829,-73.963888,3152,3 Ave & E 71 St,40.768737,-73.961199,21211,Customer,NaN,0
4,621,2017-01-01 00:01:25,2017-01-01 00:11:47,3143,5 Ave & E 78 St,40.776829,-73.963888,3152,3 Ave & E 71 St,40.768737,-73.961199,26819,Customer,NaN,0


In [22]:
df['User Type'].value_counts()

Subscriber    700465
Customer       23018
Name: User Type, dtype: int64

In [23]:
df.columns

Index(['Trip Duration', 'Start Time', 'Stop Time', 'Start Station ID',
       'Start Station Name', 'Start Station Latitude',
       'Start Station Longitude', 'End Station ID', 'End Station Name',
       'End Station Latitude', 'End Station Longitude', 'Bike ID', 'User Type',
       'Birth Year', 'Gender'],
      dtype='object')

In [25]:
df2=df.drop(['Trip Duration', 'Stop Time', 'Start Station ID',
       'Start Station Name', 'Start Station Latitude',
       'Start Station Longitude', 'End Station ID', 'End Station Name',
       'End Station Latitude', 'End Station Longitude', 'Bike ID',
       'Birth Year', 'Gender'], axis=1)

In [26]:
df2.head(5)

,Start Time,User Type
0,2017-01-01 00:00:21,Subscriber
1,2017-01-01 00:00:45,Subscriber
2,2017-01-01 00:00:57,Customer
3,2017-01-01 00:01:10,Customer
4,2017-01-01 00:01:25,Customer


In [27]:
#Total number of rides
len(df2)

726676

In [28]:
df2['User Type'].unique()

array(['Subscriber', 'Customer', nan], dtype=object)

In [29]:
totalTrips = df2.groupby('User Type')['Start Time'].nunique()
print (totalTrips)

User Type
Customer       22166
Subscriber    551328
Name: Start Time, dtype: int64


In [30]:
# Count of total riders on each day
df2['date'] = pd.to_datetime(df2['Start Time'])
totalPerDay = df2['User Type'].groupby(df2['date'].dt.weekday).count()
print (totalPerDay)

date
0     93955
1     94272
2    133796
3    137154
4    116556
5     65690
6     82060
Name: User Type, dtype: int64


In [34]:
# Count of subscribers on each day
Sub = df2['User Type'] =='Subscriber'
Sdate = df2['date'][Sub]
SNum = Sdate.groupby([df2['date'].dt.weekday]).count()
print(SNum)

date
0     91567
1     93520
2    130758
3    134052
4    114035
5     61836
6     74697
Name: date, dtype: int64


In [35]:
# Count of customers on each day
Cust = df2['User Type'] =='Customer'
Cdate = df2['date'][Cust]
CNum = Cdate.groupby([df2['date'].dt.weekday]).count()
print(CNum)

date
0    2388
1     752
2    3038
3    3102
4    2521
5    3854
6    7363
Name: date, dtype: int64


In [36]:
#Adds Sat & Sun Customer rides

TotalWkendCustomers= CNum[5] + CNum[6]

In [37]:
#Adds Sat & Sun Subsriber Rides

TotalWkendSubscribers= SNum[5] + SNum[6]

In [38]:
print('Weekend Customer Rides:', TotalWkendCustomers) 
print('Weekend Subscriber Rides:', TotalWkendSubscribers)

Weekend Customer Rides: 11217
Weekend Subscriber Rides: 136533


In [39]:
TotalWkdayCustomers= CNum[0] + CNum[1] + CNum[2] + CNum[3] + CNum[4]

In [40]:
TotalWkdaySubscribers= SNum[0] + SNum[1] +SNum[2] + SNum[3] + SNum[4]

In [41]:
print('Weekday Customer Rides:', TotalWkdayCustomers) 
print('Weekday Subscriber Rides:', TotalWkdaySubscribers)

Weekday Customer Rides: 11801
Weekday Subscriber Rides: 563932


In [42]:
# Adding Weekday number into my dataset
df2['Day'] = df2['date'].dt.weekday
df2.head()

,Start Time,User Type,date,Day
0,2017-01-01 00:00:21,Subscriber,2017-01-01 00:00:21,6
1,2017-01-01 00:00:45,Subscriber,2017-01-01 00:00:45,6
2,2017-01-01 00:00:57,Customer,2017-01-01 00:00:57,6
3,2017-01-01 00:01:10,Customer,2017-01-01 00:01:10,6
4,2017-01-01 00:01:25,Customer,2017-01-01 00:01:25,6


In [43]:
df2.Day.unique()

array([6, 0, 1, 2, 3, 4, 5])

In [44]:
df2_wd=df2[df2.Day.between(0,4)]
df2_wd['User Type'].unique()

array(['Subscriber', 'Customer', nan], dtype=object)

In [45]:
df2_wk=df2[df2.Day.between(5,6)]


In [46]:
# 3000 samples from weekday data
np.random.seed(12345)
wd=np.random.choice(df2_wd['User Type'], 3000)

In [47]:
# 3000 samples from weekend data

wk=np.random.choice(df2_wk['User Type'], 3000)

In [48]:
wkdayC=np.count_nonzero(wd=='Customer')
wkdayS=np.count_nonzero(wd=='Subscriber')
print('Weekday Customer:', wkdayC, '\nWeekday Subscriber:', wkdayS)

Weekday Customer: 70 
Weekday Subscriber: 2917


In [49]:
wkendC=np.count_nonzero(wk=='Customer')
wkendS=np.count_nonzero(wk=='Subscriber')
print('Weekend Customer:', wkendC, '\nWeekend Subscriber:', wkendS)

Weekend Customer: 227 
Weekend Subscriber: 2754


In [50]:
import numpy as np

def evalChisq(values):
    values = np.array(values)
    E = np.empty_like(values)
    for j in range(len(values[0])):
        for i in range(2):
            
            E[i][j] = ((values[i,:].sum() * values[:,j].sum()) / 
                        (values).sum())
    return ((values - E)**2 / E).sum()

In [51]:
sample = np.array([[wkdayC, wkdayS], [wkendC, wkendS]])
a= evalChisq(sample)

print (a)

87.6244159736
